# Netflix Streaming Analysis – Cleaned & Styled Notebook

This notebook:

1. Loads the raw CSV files into DuckDB.
2. Profiles missing values and duplicates for each table.
3. Cleans each table (removing/handling duplicates and missing values).
4. Builds a merged analytical view.
5. Runs exploratory data analysis (EDA) and key business insights with **enhanced visualizations**.

Adjust the `DATA_DIR` below if your CSV files are in a different folder on your machine.


In [1]:
import duckdb, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Use a nicer global style for all plots
plt.style.use("ggplot")

# Adjust this path if needed
DATA_DIR = "/Users/nelmierandikudagodage/Downloads/DW/netflix"

con = duckdb.connect(database=':memory:')

# Load CSVs into DuckDB staging tables
con.execute(f"""
    CREATE TABLE users AS
    SELECT * FROM read_csv_auto('{os.path.join(DATA_DIR, "users.csv")}');

    CREATE TABLE watch_history AS
    SELECT * FROM read_csv_auto('{os.path.join(DATA_DIR, "watch_history.csv")}');

    CREATE TABLE movies AS
    SELECT * FROM read_csv_auto('{os.path.join(DATA_DIR, "movies.csv")}');

    CREATE TABLE reviews AS
    SELECT * FROM read_csv_auto('{os.path.join(DATA_DIR, "reviews.csv")}');

    CREATE TABLE recommendation_logs AS
    SELECT * FROM read_csv_auto('{os.path.join(DATA_DIR, "recommendation_logs.csv")}');

    CREATE TABLE search_logs AS
    SELECT * FROM read_csv_auto('{os.path.join(DATA_DIR, "search_logs.csv")}');
""")

tables = [
    "users",
    "watch_history",
    "movies",
    "reviews",
    "recommendation_logs",
    "search_logs"
]

tables


['users',
 'watch_history',
 'movies',
 'reviews',
 'recommendation_logs',
 'search_logs']

## 1. Raw Data Preview

In [2]:
for t in tables:
    print(f"\nPreview of {t}:")
    df_head = con.execute(f"SELECT * FROM {t} LIMIT 5").fetchdf()
    display(df_head)



Preview of users:


,user_id,email,first_name,last_name,age,gender,country,state_province,city,subscription_plan,subscription_start_date,is_active,monthly_spend,primary_device,household_size,created_at
0,user_00001,figueroajohn@example.org,Erica,Garza,43.0,Male,USA,Massachusetts,North Jefferyhaven,Basic,2024-04-08,True,36.06,Laptop,1.0,2023-04-01 14:40:50.540242
1,user_00002,blakeerik@example.com,Joshua,Bernard,38.0,Male,USA,Texas,North Noahstad,Premium+,2024-05-24,True,14.59,Desktop,2.0,2024-10-10 15:39:11.030515
2,user_00003,smiller@example.net,Barbara,Williams,32.0,Female,USA,Michigan,Traciebury,Standard,2023-09-22,False,11.71,Desktop,3.0,2024-06-29 14:27:49.560875
3,user_00004,mitchellclark@example.com,Chelsea,Ferguson,11.0,Male,USA,Ohio,South Noah,Standard,2024-08-21,True,28.56,Laptop,2.0,2023-04-11 01:01:59.614841
4,user_00005,richard13@example.net,Jason,Foster,21.0,Female,USA,Arizona,West Donald,Standard,2024-10-28,True,9.54,Desktop,6.0,2025-04-12 19:59:30.137806



Preview of watch_history:


,session_id,user_id,movie_id,watch_date,device_type,watch_duration_minutes,progress_percentage,action,quality,location_country,is_download,user_rating
0,session_000001,user_07271,movie_0511,2025-11-13,Tablet,63.9,34.6,completed,HD,USA,False,<NA>
1,session_000002,user_00861,movie_0588,2025-02-26,Laptop,120.1,44.2,started,HD,USA,False,<NA>
2,session_000003,user_05391,movie_0694,2024-12-15,Desktop,572.1,84.7,started,HD,Canada,False,1
3,session_000004,user_05192,movie_0234,2024-09-30,Desktop,395.3,89.9,completed,SD,USA,False,5
4,session_000005,user_05735,movie_0390,2024-08-04,Tablet,14.6,6.2,completed,HD,USA,False,<NA>



Preview of movies:


...

,review_id,user_id,movie_id,rating,review_date,device_type,is_verified_watch,helpful_votes,total_votes,review_text,sentiment,sentiment_score
0,review_000001,user_07066,movie_0360,4,2025-03-29,Mobile,False,3.0,5.0,Fantastic cinematography and plot twists.,positive,0.711
1,review_000002,user_02953,movie_0095,5,2024-07-19,Mobile,True,2.0,2.0,This series is a masterpiece!,positive,0.645
2,review_000003,user_05528,movie_0518,4,2025-02-11,Tablet,True,2.0,5.0,Fantastic cinematography and plot twists.,positive,0.730
3,review_000004,user_07612,movie_0672,5,2025-11-26,Mobile,True,7.0,7.0,One of the best series I've ever watched. High...,positive,0.798
4,review_000005,user_03424,movie_0580,3,2025-07-11,Mobile,True,1.0,5.0,Mixed feelings about this one.,neutral,0.410



Preview of recommendation_logs:


,recommendation_id,user_id,movie_id,recommendation_date,recommendation_type,recommendation_score,was_clicked,position_in_list,device_type,time_of_day,algorithm_version
0,rec_000001,user_06326,movie_0771,2025-07-03,new_releases,NaN,False,10,Tablet,evening,v1.4
1,rec_000002,user_02180,movie_0985,2024-11-07,genre_based,0.916,False,9,Mobile,evening,v1.4
2,rec_000003,user_03535,movie_0834,2024-08-22,personalized,0.816,False,2,Tablet,evening,v1.4
3,rec_000004,user_05025,movie_0718,2024-04-12,trending,0.771,False,16,Mobile,evening,v1.2
4,rec_000005,user_06794,movie_0718,2025-10-30,similar_users,0.251,False,9,Tablet,evening,v1.3



Preview of search_logs:


,search_id,user_id,search_query,search_date,results_returned,clicked_result_position,device_type,search_duration_seconds,had_typo,used_filters,location_country
0,search_000001,user_09864,classic movies,2024-03-22,20,2,Tablet,12.4,False,False,Canada
1,search_000002,user_08038,stand up comedy,2025-11-22,24,4,Tablet,63.5,True,False,USA
2,search_000003,user_02009,music documentaries,2024-10-09,86,1,Tablet,24.7,True,False,USA
3,search_000004,user_01083,comedy shows,2024-12-14,70,4,Mobile,53.7,False,False,USA
4,search_000005,user_04269,movies based on true stories,2025-01-10,48,<NA>,Tablet,69.6,True,False,USA


## 2. Missing Values and Duplicate Checks (Raw Tables)

In [3]:
print("=== Total missing values per table (raw) ===")
for t in tables:
    df = con.execute(f"SELECT * FROM {t}").fetchdf()
    total_missing = int(df.isna().sum().sum())
    print(f"Missing values in {t}: {total_missing}")


=== Total missing values per table (raw) ===
Missing values in users: 4615
Missing values in watch_history: 104749
Missing values in movies: 3671
Missing values in reviews: 5624
Missing values in recommendation_logs: 7836
Missing values in search_logs: 14826


In [4]:
id_columns = {
    "users": "user_id",
    "watch_history": "session_id",
    "movies": "movie_id",
    "reviews": "review_id",
    "recommendation_logs": "recommendation_id",
    "search_logs": "search_id",
}

print("\n=== Duplicate checks by ID column (raw) ===")
for table, id_col in id_columns.items():
    total_rows, unique_ids = con.execute(f"""
        SELECT
            COUNT(*) AS total_rows,
            COUNT(DISTINCT {id_col}) AS unique_ids
        FROM {table}
    """).fetchone()
    duplicate_count = total_rows - unique_ids
    print(f"Duplicate {id_col} values in {table}: {duplicate_count}")



=== Duplicate checks by ID column (raw) ===
Duplicate user_id values in users: 300
Duplicate session_id values in watch_history: 5000
Duplicate movie_id values in movies: 40
Duplicate review_id values in reviews: 450
Duplicate recommendation_id values in recommendation_logs: 2000
Duplicate search_id values in search_logs: 1500


## 3. Cleaning Steps

### 3.1 Cleaning `users`

In [5]:
# 1) De-duplicate users on user_id (keep most recent subscription_start_date)
con.execute("""
CREATE OR REPLACE TABLE users_dedup AS
WITH ranked AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY user_id
            ORDER BY subscription_start_date DESC NULLS LAST
        ) AS rn
    FROM users
    WHERE user_id IS NOT NULL
)
SELECT * EXCLUDE (rn)
FROM ranked
WHERE rn = 1;
""")

# 2) Impute missing values in users_dedup
con.execute("""
CREATE OR REPLACE TABLE users_clean AS
WITH medians AS (
    SELECT
        median(age) AS med_age,
        median(monthly_spend) AS med_spend
    FROM users_dedup
)
SELECT
    u.user_id,
    COALESCE(u.age, m.med_age) AS age,
    COALESCE(u.gender, 'Unknown') AS gender,
    COALESCE(u.country, 'Unknown') AS country,
    COALESCE(u.subscription_plan, 'Unknown') AS subscription_plan,
    u.subscription_start_date,
    COALESCE(u.is_active, FALSE) AS is_active,
    COALESCE(u.monthly_spend, m.med_spend) AS monthly_spend
FROM users_dedup u
CROSS JOIN medians m;
""")

users_clean = con.execute("SELECT * FROM users_clean").fetchdf()
users_clean.head()


,user_id,age,gender,country,subscription_plan,subscription_start_date,is_active,monthly_spend
0,user_00010,24.0,Male,USA,Basic,2024-08-04,True,23.18
1,user_00022,33.0,Male,USA,Premium+,2023-09-05,True,13.56
2,user_00059,40.0,Unknown,USA,Standard,2024-05-06,True,13.00
3,user_00064,40.0,Female,Canada,Premium,2023-09-23,True,6.76
4,user_00077,35.0,Female,USA,Premium,2023-07-08,False,27.46


### 3.2 Cleaning `watch_history`

In [6]:
con.execute("""
CREATE OR REPLACE TABLE watch_history_clean AS
SELECT
    session_id,
    user_id,
    movie_id,
    watch_date,
    COALESCE(device_type, 'Unknown') AS device_type,
    COALESCE(watch_duration_minutes, 0) AS watch_duration_minutes,
    COALESCE(progress_percentage, 0) AS progress_percentage,
    COALESCE(action, 'Unknown') AS action,
    COALESCE(quality, 'Unknown') AS quality,
    COALESCE(location_country, 'Unknown') AS location_country,
    COALESCE(is_download, FALSE) AS is_download,
    user_rating
FROM (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY session_id
            ORDER BY watch_date DESC NULLS LAST
        ) AS rn
    FROM watch_history
)
WHERE rn = 1
  AND session_id IS NOT NULL
  AND user_id IS NOT NULL
  AND movie_id IS NOT NULL
  AND watch_date IS NOT NULL;
""")

watch_history_clean = con.execute("SELECT * FROM watch_history_clean").fetchdf()
watch_history_clean.head()


,session_id,user_id,movie_id,watch_date,device_type,watch_duration_minutes,progress_percentage,action,quality,location_country,is_download,user_rating
0,session_030582,user_00841,movie_0799,2025-04-04,Laptop,0.0,69.4,paused,4K,USA,True,<NA>
1,session_030607,user_02509,movie_0099,2025-08-06,Laptop,0.0,45.0,paused,4K,USA,False,<NA>
2,session_030633,user_08677,movie_0994,2024-03-30,Smart TV,113.9,33.4,stopped,HD,USA,False,<NA>
3,session_030670,user_08831,movie_0035,2025-05-08,Tablet,0.0,58.8,paused,4K,Canada,True,<NA>
4,session_030675,user_05971,movie_0615,2025-01-16,Tablet,0.0,0.0,completed,4K,USA,False,<NA>


### 3.3 Cleaning `movies`

In [7]:
con.execute("""
CREATE OR REPLACE TABLE movies_dedup AS
WITH ranked AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY movie_id
            ORDER BY added_to_platform DESC NULLS LAST
        ) AS rn
    FROM movies
    WHERE movie_id IS NOT NULL
)
SELECT * EXCLUDE (rn)
FROM ranked
WHERE rn = 1;
""")

con.execute("""
CREATE OR REPLACE TABLE movies_clean AS
WITH medians AS (
    SELECT
        median(duration_minutes) AS med_duration
    FROM movies_dedup
)
SELECT
    movie_id,
    COALESCE(title, 'Unknown') AS title,
    COALESCE(content_type, 'Unknown') AS content_type,
    COALESCE(genre_primary, 'Unknown') AS genre_primary,
    release_year,
    COALESCE(duration_minutes, med_duration) AS duration_minutes,
    COALESCE(rating, 'Unknown') AS rating,
    COALESCE(language, 'Unknown') AS language,
    imdb_rating,
    COALESCE(is_netflix_original, FALSE) AS is_netflix_original,
    added_to_platform
FROM movies_dedup
CROSS JOIN medians;
""")

movies_clean = con.execute("SELECT * FROM movies_clean").fetchdf()
movies_clean.head()


,movie_id,title,content_type,genre_primary,release_year,duration_minutes,rating,language,imdb_rating,is_netflix_original,added_to_platform
0,movie_0013,Dream Journey,Movie,Romance,2013,138.0,PG,English,8.6,False,2021-01-20
1,movie_0018,Princess Phoenix,Movie,Adventure,2011,116.0,PG,English,7.6,True,2021-04-04
2,movie_0034,Mission Kingdom,TV Series,War,2018,48.0,TV-PG,English,7.7,True,2023-10-22
3,movie_0037,Empire King,TV Series,Comedy,2011,51.0,TV-G,French,NaN,False,2025-05-02
4,movie_0046,The Dragon,TV Series,Family,2002,43.0,PG-13,English,6.7,False,2023-08-15


### 3.4 Cleaning `reviews`

In [8]:
con.execute("""
CREATE OR REPLACE TABLE reviews_dedup AS
WITH ranked AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY review_id
            ORDER BY review_date DESC NULLS LAST
        ) AS rn
    FROM reviews
    WHERE review_id IS NOT NULL
)
SELECT * EXCLUDE (rn)
FROM ranked
WHERE rn = 1;
""")

con.execute("""
CREATE OR REPLACE TABLE reviews_clean AS
SELECT
    review_id,
    user_id,
    movie_id,
    rating,
    review_date,
    COALESCE(is_verified_watch, FALSE) AS is_verified_watch,
    COALESCE(helpful_votes, 0) AS helpful_votes,
    COALESCE(total_votes, 0) AS total_votes,
    COALESCE(review_text, '') AS review_text,
    COALESCE(sentiment, 'Unknown') AS sentiment,
    COALESCE(sentiment_score, 0) AS sentiment_score
FROM reviews_dedup
WHERE review_id IS NOT NULL
  AND user_id IS NOT NULL
  AND movie_id IS NOT NULL
  AND rating IS NOT NULL
  AND review_date IS NOT NULL;
""")

reviews_clean = con.execute("SELECT * FROM reviews_clean").fetchdf()
reviews_clean.head()


,review_id,user_id,movie_id,rating,review_date,is_verified_watch,helpful_votes,total_votes,review_text,sentiment,sentiment_score
0,review_000001,user_07066,movie_0360,4,2025-03-29,False,3.0,5.0,Fantastic cinematography and plot twists.,positive,0.711
1,review_000046,user_05756,movie_0554,3,2024-08-29,True,0.0,0.0,Mixed feelings about this one.,neutral,0.000
2,review_000078,user_08701,movie_0135,4,2024-09-02,True,2.0,6.0,Brilliant performance by the cast. Must watch!,positive,0.752
3,review_000079,user_03228,movie_0742,5,2025-11-06,True,3.0,6.0,Brilliant performance by the cast. Must watch!,positive,0.000
4,review_000085,user_04915,movie_0269,3,2024-10-09,False,1.0,6.0,Decent watch but forgettable.,neutral,0.529


### 3.5 Cleaning `recommendation_logs`

In [9]:
con.execute("""
CREATE OR REPLACE TABLE recommendation_logs_dedup AS
WITH ranked AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY recommendation_id
            ORDER BY recommendation_date DESC NULLS LAST
        ) AS rn
    FROM recommendation_logs
    WHERE recommendation_id IS NOT NULL
)
SELECT * EXCLUDE (rn)
FROM ranked
WHERE rn = 1;
""")

con.execute("""
CREATE OR REPLACE TABLE recommendation_logs_clean AS
SELECT
    recommendation_id,
    user_id,
    movie_id,
    recommendation_date,
    COALESCE(recommendation_type, 'Unknown') AS recommendation_type,
    recommendation_score,
    COALESCE(was_clicked, FALSE) AS was_clicked,
    COALESCE(position_in_list, 0) AS position_in_list,
    COALESCE(device_type, 'Unknown') AS device_type,
    COALESCE(time_of_day, 'Unknown') AS time_of_day,
    COALESCE(algorithm_version, 'Unknown') AS algorithm_version
FROM recommendation_logs_dedup
WHERE recommendation_id IS NOT NULL
  AND user_id IS NOT NULL
  AND movie_id IS NOT NULL
  AND recommendation_date IS NOT NULL;
""")

recommendation_logs_clean = con.execute("SELECT * FROM recommendation_logs_clean").fetchdf()
recommendation_logs_clean.head()


,recommendation_id,user_id,movie_id,recommendation_date,recommendation_type,recommendation_score,was_clicked,position_in_list,device_type,time_of_day,algorithm_version
0,rec_000008,user_03328,movie_0385,2024-10-09,personalized,0.755,False,11,Tablet,night,v1.4
1,rec_000021,user_03307,movie_0087,2025-04-13,personalized,NaN,False,15,Tablet,afternoon,v1.3
2,rec_000073,user_06552,movie_0652,2025-12-29,trending,0.538,False,13,Tablet,evening,v2.0
3,rec_000076,user_04204,movie_0988,2025-04-20,new_releases,0.125,False,4,Mobile,night,Unknown
4,rec_000099,user_08274,movie_0355,2024-01-23,genre_based,0.188,False,9,Smart TV,morning,v2.0


### 3.6 Cleaning `search_logs`

In [10]:
con.execute("""
CREATE OR REPLACE TABLE search_logs_dedup AS
WITH ranked AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY search_id
            ORDER BY search_date DESC NULLS LAST
        ) AS rn
    FROM search_logs
    WHERE search_id IS NOT NULL
)
SELECT * EXCLUDE (rn)
FROM ranked
WHERE rn = 1;
""")

con.execute("""
CREATE OR REPLACE TABLE search_logs_clean AS
SELECT
    search_id,
    user_id,
    search_query,
    search_date,
    COALESCE(results_returned, 0) AS results_returned,
    COALESCE(clicked_result_position, 0) AS clicked_result_position,
    COALESCE(device_type, 'Unknown') AS device_type,
    COALESCE(search_duration_seconds, 0) AS search_duration_seconds,
    COALESCE(had_typo, FALSE) AS had_typo,
    COALESCE(used_filters, FALSE) AS used_filters,
    COALESCE(location_country, 'Unknown') AS location_country
FROM search_logs_dedup
WHERE search_id IS NOT NULL
  AND user_id IS NOT NULL
  AND search_date IS NOT NULL;
""")

search_logs_clean = con.execute("SELECT * FROM search_logs_clean").fetchdf()
search_logs_clean.head()


,search_id,user_id,search_query,search_date,results_returned,clicked_result_position,device_type,search_duration_seconds,had_typo,used_filters,location_country
0,search_000034,user_09443,cooking shows,2024-07-13,100,0,Laptop,11.8,False,False,USA
1,search_000082,user_09717,short episodes under 30 minutes,2025-12-28,70,1,Laptop,17.6,False,False,USA
2,search_000107,user_06771,biographies,2025-08-20,80,1,Smart TV,10.1,True,True,USA
3,search_000118,user_03644,classic movies,2025-01-10,84,0,Tablet,9.8,False,True,USA
4,search_000135,user_02611,sci-fi,2024-07-01,43,7,Mobile,22.5,False,False,USA


### 3.7 Quality Check After Cleaning

In [11]:
clean_tables = [
    "users_clean",
    "watch_history_clean",
    "movies_clean",
    "reviews_clean",
    "recommendation_logs_clean",
    "search_logs_clean"
]

print("=== Total missing values per table (cleaned) ===")
for t in clean_tables:
    df = con.execute(f"SELECT * FROM {t}").fetchdf()
    total_missing = int(df.isna().sum().sum())
    print(f"Missing values in {t}: {total_missing}")

print("\n=== Duplicate checks by ID column (cleaned) ===")
clean_id_columns = {
    "users_clean": "user_id",
    "watch_history_clean": "session_id",
    "movies_clean": "movie_id",
    "reviews_clean": "review_id",
    "recommendation_logs_clean": "recommendation_id",
    "search_logs_clean": "search_id",
}

for table, id_col in clean_id_columns.items():
    total_rows, unique_ids = con.execute(f"""
        SELECT
            COUNT(*) AS total_rows,
            COUNT(DISTINCT {id_col}) AS unique_ids
        FROM {table}
    """).fetchone()
    duplicate_count = total_rows - unique_ids
    print(f"Duplicate {id_col} values in {table}: {duplicate_count}")


=== Total missing values per table (cleaned) ===
Missing values in users_clean: 0
Missing values in watch_history_clean: 79886
Missing values in movies_clean: 144
Missing values in reviews_clean: 0
Missing values in recommendation_logs_clean: 5005
Missing values in search_logs_clean: 0

=== Duplicate checks by ID column (cleaned) ===
Duplicate user_id values in users_clean: 0
Duplicate session_id values in watch_history_clean: 0
Duplicate movie_id values in movies_clean: 0
Duplicate review_id values in reviews_clean: 0
Duplicate recommendation_id values in recommendation_logs_clean: 0
Duplicate search_id values in search_logs_clean: 0


## 4. Merged Analytical View

In [12]:
con.execute("""
CREATE OR REPLACE VIEW vw_user_movie_sessions AS
SELECT
    w.session_id,
    w.user_id,
    w.movie_id,
    w.watch_date,
    w.watch_duration_minutes,
    w.progress_percentage,
    w.device_type         AS watch_device,
    w.location_country    AS watch_country,
    w.is_download,
    w.user_rating,
    -- user attributes
    u.age,
    u.gender,
    u.country             AS user_country,
    u.subscription_plan,
    u.monthly_spend,
    u.is_active,
    -- movie attributes
    m.title,
    m.content_type,
    m.genre_primary,
    m.release_year,
    m.duration_minutes    AS movie_duration_minutes,
    m.rating              AS content_rating,
    m.language,
    m.imdb_rating,
    m.is_netflix_original
FROM watch_history_clean w
LEFT JOIN users_clean  u USING (user_id)
LEFT JOIN movies_clean m USING (movie_id);
""")

con.execute("SELECT * FROM vw_user_movie_sessions LIMIT 5").fetchdf()


,session_id,user_id,movie_id,watch_date,watch_duration_minutes,progress_percentage,watch_device,watch_country,is_download,user_rating,...,is_active,title,content_type,genre_primary,release_year,movie_duration_minutes,content_rating,language,imdb_rating,is_netflix_original
0,session_030582,user_00841,movie_0799,2025-04-04,0.0,69.4,Laptop,USA,True,<NA>,...,True,Fire Secret,TV Series,Crime,1992,39.0,TV-PG,English,8.3,True
1,session_030607,user_02509,movie_0099,2025-08-06,0.0,45.0,Laptop,USA,False,<NA>,...,True,Dragon Day,Documentary,Western,2021,110.0,PG,English,8.3,False
2,session_030633,user_08677,movie_0994,2024-03-30,113.9,33.4,Smart TV,USA,False,<NA>,...,True,Battle Queen,Movie,Mystery,1990,119.0,TV-14,German,1.4,False
3,session_030670,user_08831,movie_0035,2025-05-08,0.0,58.8,Tablet,Canada,True,<NA>,...,True,Princess Love,TV Series,Family,1994,38.0,TV-PG,English,7.4,False
4,session_030675,user_05971,movie_0615,2025-01-16,0.0,0.0,Tablet,USA,False,<NA>,...,True,An Fire,Movie,Crime,2016,136.0,PG,English,5.1,True


## 5. Exploratory Data Analysis (EDA) – Styled Visuals

### 5.1 Users – Demographics and Plans

In [13]:
users_clean = con.execute("SELECT * FROM users_clean").fetchdf()
display(users_clean.describe(include="all"))

# Age distribution
plt.figure(figsize=(8, 5))
plt.hist(users_clean['age'].dropna(), bins=20, color="#4C72B0", edgecolor="white", alpha=0.85)
plt.title("Age Distribution of Users", fontsize=14)
plt.xlabel("Age")
plt.ylabel("Number of Users")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Monthly spend distribution
plt.figure(figsize=(8, 5))
plt.hist(users_clean['monthly_spend"].dropna(), bins=20, color="#55A868", edgecolor="white", alpha=0.85)
plt.title("Monthly Spend Distribution", fontsize=14)
plt.xlabel("Monthly Spend")
plt.ylabel("Number of Users")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Top subscription plans
plan_counts = users_clean['subscription_plan'].value_counts().head(10)
plt.figure(figsize=(9, 5))
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(plan_counts)))
plt.bar(plan_counts.index, plan_counts.values, color=colors)
plt.title("Top Subscription Plans", fontsize=14)
plt.xlabel("Subscription Plan")
plt.ylabel("Number of Users")
plt.xticks(rotation=30, ha="right")
for i, v in enumerate(plan_counts.values):
    plt.text(i, v + max(plan_counts.values)*0.01, str(v), ha="center", va="bottom", fontsize=8)
plt.tight_layout()
plt.show()


SyntaxError: unterminated string literal (detected at line 16) (2416826474.py, line 16)

## 7. Summary

- Raw CSVs were loaded into DuckDB staging tables.
- Missing values and duplicates were profiled and then systematically cleaned.
- Cleaned tables (`*_clean`) enforce one row per business key and handle missing numeric and categorical values.
- A merged analytical view (`vw_user_movie_sessions`) supports cross-cutting analysis of users, content, and behaviour.
- EDA and key business insights are visualised with styled, presentation-ready charts.
